In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Nhập các gói thư viện cần thiết**

In [ ]:
# import packages
# thu vien doc du lieu bang
import pandas as pd
# thu vien in hình anh 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# thu vien tinh toan 
from scipy.stats import norm
from scipy import stats
import warnings
from sklearn.ensemble import RandomForestRegressor
warnings.filterwarnings('ignore')
%matplotlib inline
# thu vien chia Kfold
from sklearn.model_selection import KFold
# thu vien chia tinh toan mean_square_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
#impute
from sklearn.impute import SimpleImputer

In [ ]:
#đọc dữ liệu từ input
train_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
test_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')
test_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')

# Xem xét về tập train và test

**1. Kích thước và các trường dữ liệu**

In [ ]:
#train transaction
print(train_transaction.shape)
print(train_transaction.columns)

In [ ]:
#train idenitity
print(train_identity.shape)
print(train_identity.columns)

In [ ]:
#test transaction
print(test_transaction.shape)
print(test_transaction.columns)

In [ ]:
# test identity
print(test_identity.shape)
print(test_identity.columns)

**2. Ghép các tập dữ liệu theo TransactionID**

In [ ]:
#train data
train = pd.merge(train_transaction, train_identity, how = 'left', on = 'TransactionID')
train = train.drop(columns = 'TransactionID')
train.shape

In [ ]:
#test data
test = pd.merge(test_transaction, test_identity, how = 'left', on = 'TransactionID')
test = test.drop(columns = 'TransactionID')
print(test.shape)


**Rename**

In [ ]:
test = test.rename(columns = {"id-01": "id_01", "id-02": "id_02", "id-03": "id_03", 
                            "id-06": "id_06", "id-05": "id_05", "id-04": "id_04", 
                            "id-07": "id_07", "id-08": "id_08", "id-09": "id_09", 
                            "id-10": "id_10", "id-11": "id_11", "id-12": "id_12", 
                            "id-15": "id_15", "id-14": "id_14", "id-13": "id_13", 
                            "id-16": "id_16", "id-17": "id_17", "id-18": "id_18", 
                            "id-21": "id_21", "id-20": "id_20", "id-19": "id_19", 
                            "id-22": "id_22", "id-23": "id_23", "id-24": "id_24", 
                            "id-27": "id_27", "id-26": "id_26", "id-25": "id_25", 
                            "id-28": "id_28", "id-29": "id_29", "id-30": "id_30", 
                            "id-31": "id_31", "id-32": "id_32", "id-33": "id_33", 
                            "id-34": "id_34", "id-35": "id_35", "id-36": "id_36", 
                            "id-37": "id_37", "id-38": "id_38"})
test.columns

**3. Reduce memory usage**

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = reduce_mem_usage(train, verbose = True)
test = reduce_mem_usage(test, verbose = True)

In [ ]:
del train_transaction, train_identity, test_transaction, test_identity

In [ ]:
def detect_num_cols_to_shrink(list_of_num_cols, dataframe):
 
    convert_to_int8 = []
    convert_to_int16 = []
    convert_to_int32 = []
    
    #  sadly the datatype float8 does not exist
    convert_to_float16 = []
    convert_to_float32 = []
    
    for col in list_of_num_cols:
        
        if dataframe[col].dtype in ['int', 'int8', 'int32', 'int64']:
            
            describe_object = dataframe[col].describe()
            minimum = describe_object[3]
            maximum = describe_object[7]
            diff = abs(maximum - minimum)

            if diff < 255:
                convert_to_int8.append(col)
                
            elif diff < 65535:
                convert_to_int16.append(col)
                
            elif diff < 4294967295:
                convert_to_int32.append(col)   
                
        elif dataframe[col].dtype in ['float', 'float16', 'float32', 'float64']:
            
            describe_object = dataframe[col].describe()
            minimum = describe_object[3]
            maximum = describe_object[7]
            diff = abs(maximum - minimum)

            if diff < 65535:
                convert_to_float16.append(col)
                
            elif diff < 4294967295:
                convert_to_float32.append(col) 
        
    list_of_lists = []
    list_of_lists.append(convert_to_int8)
    list_of_lists.append(convert_to_int16)
    list_of_lists.append(convert_to_int32)
    list_of_lists.append(convert_to_float16)
    list_of_lists.append(convert_to_float32)
    
    return list_of_lists

In [ ]:
n = (train.dtypes != 'object')
num_train_cols = list(n[n].index) 

In [ ]:
num_cols_to_shrink_train = detect_num_cols_to_shrink(num_train_cols, train)

convert_to_int8 = num_cols_to_shrink_train[0]
convert_to_int16 = num_cols_to_shrink_train[1]
convert_to_int32 = num_cols_to_shrink_train[2]

convert_to_float16 = num_cols_to_shrink_train[3]
convert_to_float32 = num_cols_to_shrink_train[4]

print("Train")

print("convert_to_int8 :", convert_to_int8, "\n")
print("convert_to_int16 :", convert_to_int16, "\n")
print("convert_to_int32 :", convert_to_int32, "\n")

print("convert_to_float16 :", convert_to_float16, "\n")
print("convert_to_float32 :", convert_to_float32, "\n")

In [ ]:
n = (test.dtypes != 'object')
num_test_cols = list(n[n].index) 

In [ ]:
num_cols_to_shrink_test = detect_num_cols_to_shrink(num_test_cols, test)

convert_to_int8 = num_cols_to_shrink_test[0]
convert_to_int16 = num_cols_to_shrink_test[1]
convert_to_int32 = num_cols_to_shrink_test[2]

convert_to_float16 = num_cols_to_shrink_test[3]
convert_to_float32 = num_cols_to_shrink_test[4]

print("Test")

print("convert_to_int8 :", convert_to_int8, "\n")
print("convert_to_int16 :", convert_to_int16, "\n")
print("convert_to_int32 :", convert_to_int32, "\n")

print("convert_to_float16 :", convert_to_float16, "\n")
print("convert_to_float32 :", convert_to_float32, "\n")

# Giới thiệu về data

## Thông tin các trường dữ liệu

In [ ]:
print("Kích thước tập train: ",train.shape)
print("Kích thước tập test: ",test.shape)

In [ ]:
#các trường dữ liệu
train.columns

In [ ]:
train.head(5)

## Target

In [ ]:
#plot fraud and not fraud
plt_target = plt.figure(figsize = (6,6))
sns.barplot([0,1],train['isFraud'].value_counts().values)
plt.show()

# Khai phá dữ liệu

In [ ]:
#các trường dữ liệu 
train.dtypes.value_counts()

## Missing data

In [ ]:
def missing_values_table(df):
    # Total missing values
    mis_val = df.isnull().sum()       
    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)       
    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)        
    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})        
    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    return mis_val_table_ren_columns

In [ ]:
missing_values_table(train).head(15)

# Feature Engineering

## Kết hợp tập train và test thành data

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]
target = train.isFraud.values
data = pd.concat((train, test)).reset_index(drop=True)
data.drop(['isFraud'], axis=1, inplace=True)
print("data size is : {}".format(data.shape))

In [ ]:
del train, test

## Xử lý missing data

*1. Bỏ các cột có missing data >15%*

In [ ]:
data = data.loc[:, data.isnull().mean() < .15]
data.shape

In [ ]:
#bang missing value moi
missing_values_table(data).head(15)

*2. Xử lý các cột có missing data nhỏ*
*Fill nan bằng giá trị có tần suất xuất hiện nhiều nhất*

In [ ]:
min_data  = data.loc[:, data.isnull().mean() < .08]
missing_values_table(min_data).head(10)
for feature in min_data.columns:
    data[feature] = data[feature].fillna(data[feature].mode()[0])

*3. Các trường còn lại chia thành numerical và categorical*

*3.1 Numerical*

In [ ]:
numerical_nan = data[missing_values_table(data).index].select_dtypes(include=[np.number])
numerical_nan

In [ ]:
for feature in numerical_nan.columns:
    data[feature] = data[feature].fillna(data[feature].mode()[0])

*3.2 Categorical*

In [ ]:
categorical_nan = data[missing_values_table(data).index].select_dtypes(exclude=[np.number])
categorical_nan

In [ ]:
for feature in categorical_nan.columns:
    data[feature] = data[feature].fillna(data[feature].mode()[0])

In [ ]:
#kiểm tra lại nan table
missing_values_table(data)

## Xử lý dữ liệu dạng số

*1. Normalise data*

In [ ]:
data.head(10)

In [ ]:
feature = data.columns

In [ ]:
numerical = data.select_dtypes(include=[np.number])
numerical.columns

In [ ]:
#import một số thư viện cần thiết
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# normalizing the features
for feature in numerical:
    a = data.loc[:,feature].values
    a = np.reshape(a, (-1,1))
    a = StandardScaler().fit_transform(a)
    data.loc[:,feature]=a
data

## Xử lý dữ liệu phân loại

In [ ]:
categorical = data.select_dtypes(exclude=[np.number])
categorical.columns

In [ ]:
data.shape

In [ ]:
data = pd.get_dummies(data)
print(data.shape)

In [ ]:
data

## Tách cột train và test

In [ ]:
#tách train và test
train_set = data[:ntrain]
test_set = data[ntrain:]
print(train_set.shape)
print(test_set.shape)

In [ ]:
#kiểm tra lại target
target

In [ ]:
#xóa data frame data
del data

In [ ]:
missing_values_table(train_set)

In [ ]:
#import PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
df_pca = pd.DataFrame(pca.fit_transform(train_set))
df_pca

In [ ]:
pca.explained_variance_ratio_.sum()


In [ ]:
for i in range(20):
    df_pca.rename(columns = {i:'Feature '+str( i)}, inplace = True)
df_pca

In [ ]:
df_pca_test = pd.DataFrame(pca.fit_transform(test_set))
df_pca_test

## Xây dựng model

In [ ]:
#import 1 số thư viện cần thiết
from catboost import CatBoostClassifier, Pool
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder

*1. Chia tập train và tập test*

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_set, target, test_size=0.2)

In [ ]:
from xgboost import XGBRegressor
model = xgb.XGBClassifier(n_estimators = 2000,
                            max_depth = 12,
                            learning_rate = 0.02,
                            subsample = 0.8,
                            colsample_bytree = 0.4,
                            missing = -1,
                            random_state = 42,
                           tree_method = 'gpu_hist'
                           )
# my_pipeline = Pipeline(steps=[('preprocess',preprocessor),('model',model)])

In [ ]:
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import roc_auc_score
y_predict =  model.predict(X_valid)
auc = roc_auc_score(y_valid, y_predict)
print(auc)

In [ ]:
print(y_predict)
print(y_predict.sum())

In [ ]:
test_predict = model.predict(test_set)
test_predict

In [ ]:
test_predict.sum()

In [ ]:
test_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')
id_test = test_transaction['TransactionID']
id_test

In [ ]:
my_submission = pd.DataFrame({'TransactionID': id_test, 'isFraud': test_predict})
# you could use any filename. We choose submission here
my_submission.to_csv('./submission.csv', index=False)

In [ ]:
my_submission